# CIFAR-10 Linear Model

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- ConvNet(s)
- ResNets / HighwayNets 
- RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.
    
----

### Méthodologie
1. Créer un modèle classique.
2. Entraîner le modèle pendant 500 epochs.
3. Examiner sa courbe TensorBoard et son accuracy.
4. Augmenter un des hyperparamètre.
5. Réduire ce même hyperparamètre.
6. Dire l'influence de cette hyperparamètre sur la courbe et sur le modèle de manière générale.
7. Recommencer à partir de l'étape 3 pour tout les hyperparamètres possibles.

#### Hyperparamètres
- Batch size
- Learning rate
- Momentum
- Structure
- Fonction d'activation
- Initialization de kernel
- Regularizers (Dropout, L1 Norm, L2 Norm)

#### Qui s'occupe de quoi ?
- Perceptron (Mamadian)
- MLP (Réda M.)
- ConvNet (Reda B.)

In [9]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.activations import relu, softmax, tanh, sigmoid
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model , Sequential
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [10]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.6.0
Nom du GPU : /device:GPU:0


## Importation du dataset

In [11]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [12]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [13]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [14]:
LOG_DIR = os.path.join("logs") 
MODEL_DIR = os.path.join("models") 

## Fixer les seeds

In [15]:
set_seed(42) # TensorFlow
seed(42) # NumPy

# Modèle ResNets

##### Learning rate et momentum

In [16]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = [0.5,0.1, 0.01, 0.001]
momentums = [0.25, 0.5, 0.9]

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)

for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32 for x in range(1, num_layer+1)]

        resnet = ResNet(num_layer, filters_by_layers)
        resnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                        f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
        RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                        f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
        resnet.fit(x_train,
                        y_train,
                        batch_size=BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=(x_test, y_test),
                        shuffle=SHUFFLE,
                        callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
                       ) 
        resnet.save(RESNET_MODEL)

Epoch 1/200
98/98 [==============================] - 6s 57ms/step - loss: nan - categorical_accuracy: 0.1003 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 2/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 3/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 4/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 5/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 6/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 7/200
98/98 [=====================

Epoch 52/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 53/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 54/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 55/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 56/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 57/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 58/200
98/98 [==============

Epoch 103/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 104/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 105/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 106/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 107/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 108/200
98/98 [==============================] - 5s 54ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 109/200
98/98 [=======

Epoch 4/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 5/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 6/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 7/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 8/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 9/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 10/200
98/98 [====================

Epoch 55/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 56/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 57/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 58/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 59/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 60/200
98/98 [==============================] - 3s 33ms/step - loss: nan - categorical_accuracy: 0.1000 - val_loss: nan - val_categorical_accuracy: 0.1000
Epoch 61/200
98/98 [==============

...

Epoch 170/200
98/98 [==============================] - 5s 56ms/step - loss: 1.2681 - categorical_accuracy: 0.5528 - val_loss: 1.3148 - val_categorical_accuracy: 0.5338
Epoch 171/200
98/98 [==============================] - 5s 56ms/step - loss: 1.2658 - categorical_accuracy: 0.5553 - val_loss: 1.3039 - val_categorical_accuracy: 0.5387
Epoch 172/200
98/98 [==============================] - 5s 56ms/step - loss: 1.2621 - categorical_accuracy: 0.5555 - val_loss: 1.3434 - val_categorical_accuracy: 0.5183
Epoch 173/200
98/98 [==============================] - 5s 56ms/step - loss: 1.2610 - categorical_accuracy: 0.5567 - val_loss: 1.3020 - val_categorical_accuracy: 0.5396
Epoch 174/200
98/98 [==============================] - 5s 56ms/step - loss: 1.2592 - categorical_accuracy: 0.5587 - val_loss: 1.3553 - val_categorical_accuracy: 0.5187
Epoch 175/200
98/98 [==============================] - 5s 56ms/step - loss: 1.2583 - categorical_accuracy: 0.5557 - val_loss: 1.3042 - val_categorical_accuracy:

Epoch 19/200
98/98 [==============================] - 6s 56ms/step - loss: 1.4766 - categorical_accuracy: 0.4791 - val_loss: 1.4806 - val_categorical_accuracy: 0.4742
Epoch 20/200
98/98 [==============================] - 5s 56ms/step - loss: 1.4691 - categorical_accuracy: 0.4818 - val_loss: 1.4728 - val_categorical_accuracy: 0.4816
Epoch 21/200
98/98 [==============================] - 5s 56ms/step - loss: 1.4613 - categorical_accuracy: 0.4830 - val_loss: 1.4684 - val_categorical_accuracy: 0.4828
Epoch 22/200
98/98 [==============================] - 5s 56ms/step - loss: 1.4524 - categorical_accuracy: 0.4869 - val_loss: 1.4496 - val_categorical_accuracy: 0.4913
Epoch 23/200
98/98 [==============================] - 5s 56ms/step - loss: 1.4462 - categorical_accuracy: 0.4898 - val_loss: 1.4566 - val_categorical_accuracy: 0.4874
Epoch 24/200
98/98 [==============================] - 5s 56ms/step - loss: 1.4348 - categorical_accuracy: 0.4942 - val_loss: 1.4502 - val_categorical_accuracy: 0.485

98/98 [==============================] - 5s 56ms/step - loss: 1.1247 - categorical_accuracy: 0.6056 - val_loss: 1.2050 - val_categorical_accuracy: 0.5686
Epoch 69/200
98/98 [==============================] - 5s 56ms/step - loss: 1.1173 - categorical_accuracy: 0.6091 - val_loss: 1.2103 - val_categorical_accuracy: 0.5667
Epoch 70/200
98/98 [==============================] - 5s 56ms/step - loss: 1.1161 - categorical_accuracy: 0.6074 - val_loss: 1.2161 - val_categorical_accuracy: 0.5681
Epoch 71/200
98/98 [==============================] - 6s 57ms/step - loss: 1.1107 - categorical_accuracy: 0.6104 - val_loss: 1.1912 - val_categorical_accuracy: 0.5713
Epoch 72/200
98/98 [==============================] - 6s 57ms/step - loss: 1.1035 - categorical_accuracy: 0.6124 - val_loss: 1.2016 - val_categorical_accuracy: 0.5713
Epoch 73/200
98/98 [==============================] - 5s 56ms/step - loss: 1.0976 - categorical_accuracy: 0.6149 - val_loss: 1.2070 - val_categorical_accuracy: 0.5662
Epoch 74/20

98/98 [==============================] - 5s 56ms/step - loss: 0.8448 - categorical_accuracy: 0.7072 - val_loss: 1.1016 - val_categorical_accuracy: 0.6122
Epoch 118/200
98/98 [==============================] - 6s 57ms/step - loss: 0.8460 - categorical_accuracy: 0.7055 - val_loss: 1.0757 - val_categorical_accuracy: 0.6281
Epoch 119/200
98/98 [==============================] - 5s 56ms/step - loss: 0.8337 - categorical_accuracy: 0.7108 - val_loss: 1.1003 - val_categorical_accuracy: 0.6203
Epoch 120/200
98/98 [==============================] - 6s 56ms/step - loss: 0.8311 - categorical_accuracy: 0.7125 - val_loss: 1.0754 - val_categorical_accuracy: 0.6295
Epoch 121/200
98/98 [==============================] - 5s 56ms/step - loss: 0.8283 - categorical_accuracy: 0.7140 - val_loss: 1.0852 - val_categorical_accuracy: 0.6309
Epoch 122/200
98/98 [==============================] - 6s 56ms/step - loss: 0.8208 - categorical_accuracy: 0.7142 - val_loss: 1.0774 - val_categorical_accuracy: 0.6291
Epoch 

Epoch 166/200
98/98 [==============================] - 5s 56ms/step - loss: 0.5927 - categorical_accuracy: 0.7996 - val_loss: 1.1746 - val_categorical_accuracy: 0.6291
Epoch 167/200
98/98 [==============================] - 5s 56ms/step - loss: 0.5891 - categorical_accuracy: 0.8001 - val_loss: 1.1603 - val_categorical_accuracy: 0.6299
Epoch 168/200
98/98 [==============================] - 5s 56ms/step - loss: 0.5906 - categorical_accuracy: 0.7999 - val_loss: 1.1550 - val_categorical_accuracy: 0.6341
Epoch 169/200
98/98 [==============================] - 5s 56ms/step - loss: 0.5869 - categorical_accuracy: 0.7989 - val_loss: 1.1722 - val_categorical_accuracy: 0.6280
Epoch 170/200
98/98 [==============================] - 5s 56ms/step - loss: 0.5731 - categorical_accuracy: 0.8042 - val_loss: 1.1607 - val_categorical_accuracy: 0.6356
Epoch 171/200
98/98 [==============================] - 5s 56ms/step - loss: 0.5796 - categorical_accuracy: 0.8031 - val_loss: 1.1630 - val_categorical_accuracy:

##### Fonction d'activation 
Tanh Relu Sigmoid

In [17]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums =  0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=tanh,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

Epoch 1/200
98/98 [==============================] - 4s 40ms/step - loss: 2.1189 - categorical_accuracy: 0.2564 - val_loss: 1.9663 - val_categorical_accuracy: 0.3271
Epoch 2/200
98/98 [==============================] - 4s 36ms/step - loss: 1.9124 - categorical_accuracy: 0.3400 - val_loss: 1.8664 - val_categorical_accuracy: 0.3539
Epoch 3/200
98/98 [==============================] - 3s 36ms/step - loss: 1.8440 - categorical_accuracy: 0.3660 - val_loss: 1.8215 - val_categorical_accuracy: 0.3749
Epoch 4/200
98/98 [==============================] - 3s 36ms/step - loss: 1.8024 - categorical_accuracy: 0.3792 - val_loss: 1.7837 - val_categorical_accuracy: 0.3781
Epoch 5/200
98/98 [==============================] - 3s 36ms/step - loss: 1.7745 - categorical_accuracy: 0.3888 - val_loss: 1.7599 - val_categorical_accuracy: 0.3947
Epoch 6/200
98/98 [==============================] - 3s 36ms/step - loss: 1.7531 - categorical_accuracy: 0.3970 - val_loss: 1.7462 - val_categorical_accuracy: 0.3961
Epoc

98/98 [==============================] - 3s 35ms/step - loss: 1.1080 - categorical_accuracy: 0.6139 - val_loss: 1.2393 - val_categorical_accuracy: 0.5626
Epoch 100/200
98/98 [==============================] - 3s 35ms/step - loss: 1.1042 - categorical_accuracy: 0.6154 - val_loss: 1.2384 - val_categorical_accuracy: 0.5591
Epoch 101/200
98/98 [==============================] - 3s 36ms/step - loss: 1.1050 - categorical_accuracy: 0.6150 - val_loss: 1.2276 - val_categorical_accuracy: 0.5626
Epoch 102/200
98/98 [==============================] - 3s 35ms/step - loss: 1.1005 - categorical_accuracy: 0.6148 - val_loss: 1.2291 - val_categorical_accuracy: 0.5641
Epoch 103/200
98/98 [==============================] - 3s 35ms/step - loss: 1.0954 - categorical_accuracy: 0.6174 - val_loss: 1.2314 - val_categorical_accuracy: 0.5637
Epoch 104/200
98/98 [==============================] - 3s 35ms/step - loss: 1.0946 - categorical_accuracy: 0.6194 - val_loss: 1.2377 - val_categorical_accuracy: 0.5607
Epoch 

Epoch 148/200
98/98 [==============================] - 3s 35ms/step - loss: 0.9741 - categorical_accuracy: 0.6650 - val_loss: 1.1687 - val_categorical_accuracy: 0.5884
Epoch 149/200
98/98 [==============================] - 3s 35ms/step - loss: 0.9732 - categorical_accuracy: 0.6648 - val_loss: 1.1650 - val_categorical_accuracy: 0.5869
Epoch 150/200
98/98 [==============================] - 3s 35ms/step - loss: 0.9683 - categorical_accuracy: 0.6660 - val_loss: 1.1617 - val_categorical_accuracy: 0.5863
Epoch 151/200
98/98 [==============================] - 3s 35ms/step - loss: 0.9661 - categorical_accuracy: 0.6687 - val_loss: 1.1631 - val_categorical_accuracy: 0.5878
Epoch 152/200
98/98 [==============================] - 3s 35ms/step - loss: 0.9623 - categorical_accuracy: 0.6690 - val_loss: 1.1754 - val_categorical_accuracy: 0.5847
Epoch 153/200
98/98 [==============================] - 3s 35ms/step - loss: 0.9603 - categorical_accuracy: 0.6692 - val_loss: 1.1606 - val_categorical_accuracy:

Epoch 197/200
98/98 [==============================] - 3s 35ms/step - loss: 0.8375 - categorical_accuracy: 0.7155 - val_loss: 1.1169 - val_categorical_accuracy: 0.6088
Epoch 198/200
98/98 [==============================] - 3s 35ms/step - loss: 0.8353 - categorical_accuracy: 0.7182 - val_loss: 1.1176 - val_categorical_accuracy: 0.6078
Epoch 199/200
98/98 [==============================] - 3s 35ms/step - loss: 0.8320 - categorical_accuracy: 0.7182 - val_loss: 1.1116 - val_categorical_accuracy: 0.6109
Epoch 200/200
98/98 [==============================] - 3s 35ms/step - loss: 0.8302 - categorical_accuracy: 0.7178 - val_loss: 1.1134 - val_categorical_accuracy: 0.6107


In [18]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=sigmoid,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

Epoch 1/200
98/98 [==============================] - 6s 62ms/step - loss: 2.3587 - categorical_accuracy: 0.1021 - val_loss: 2.3697 - val_categorical_accuracy: 0.1000
Epoch 2/200
98/98 [==============================] - 6s 59ms/step - loss: 2.3345 - categorical_accuracy: 0.1058 - val_loss: 2.3459 - val_categorical_accuracy: 0.1000
Epoch 3/200
98/98 [==============================] - 6s 59ms/step - loss: 2.3369 - categorical_accuracy: 0.1069 - val_loss: 2.3247 - val_categorical_accuracy: 0.1000
Epoch 4/200
98/98 [==============================] - 6s 59ms/step - loss: 2.3191 - categorical_accuracy: 0.1131 - val_loss: 2.3284 - val_categorical_accuracy: 0.1000
Epoch 5/200
98/98 [==============================] - 6s 59ms/step - loss: 2.3202 - categorical_accuracy: 0.1159 - val_loss: 2.3041 - val_categorical_accuracy: 0.1425
Epoch 6/200
98/98 [==============================] - 6s 59ms/step - loss: 2.3075 - categorical_accuracy: 0.1220 - val_loss: 2.2856 - val_categorical_accuracy: 0.1481
Epoc

98/98 [==============================] - 6s 62ms/step - loss: 1.8128 - categorical_accuracy: 0.3699 - val_loss: 1.8285 - val_categorical_accuracy: 0.3572
Epoch 100/200
98/98 [==============================] - 6s 62ms/step - loss: 1.8109 - categorical_accuracy: 0.3709 - val_loss: 1.8117 - val_categorical_accuracy: 0.3684
Epoch 101/200
98/98 [==============================] - 6s 62ms/step - loss: 1.8042 - categorical_accuracy: 0.3761 - val_loss: 1.8019 - val_categorical_accuracy: 0.3772
Epoch 102/200
98/98 [==============================] - 6s 62ms/step - loss: 1.8093 - categorical_accuracy: 0.3726 - val_loss: 1.8147 - val_categorical_accuracy: 0.3692
Epoch 103/200
98/98 [==============================] - 6s 62ms/step - loss: 1.8006 - categorical_accuracy: 0.3760 - val_loss: 1.8022 - val_categorical_accuracy: 0.3720
Epoch 104/200
98/98 [==============================] - 6s 62ms/step - loss: 1.8017 - categorical_accuracy: 0.3755 - val_loss: 1.8116 - val_categorical_accuracy: 0.3609
Epoch 

Epoch 148/200
98/98 [==============================] - 6s 59ms/step - loss: 1.7382 - categorical_accuracy: 0.3967 - val_loss: 1.7369 - val_categorical_accuracy: 0.3945
Epoch 149/200
98/98 [==============================] - 6s 59ms/step - loss: 1.7363 - categorical_accuracy: 0.3962 - val_loss: 1.7480 - val_categorical_accuracy: 0.3965
Epoch 150/200
98/98 [==============================] - 6s 59ms/step - loss: 1.7351 - categorical_accuracy: 0.3981 - val_loss: 1.7348 - val_categorical_accuracy: 0.3999
Epoch 151/200
98/98 [==============================] - 6s 59ms/step - loss: 1.7307 - categorical_accuracy: 0.4003 - val_loss: 1.7411 - val_categorical_accuracy: 0.4000
Epoch 152/200
98/98 [==============================] - 6s 59ms/step - loss: 1.7276 - categorical_accuracy: 0.4003 - val_loss: 1.7416 - val_categorical_accuracy: 0.3985
Epoch 153/200
98/98 [==============================] - 6s 59ms/step - loss: 1.7314 - categorical_accuracy: 0.3991 - val_loss: 1.7295 - val_categorical_accuracy:

Epoch 197/200
98/98 [==============================] - 6s 64ms/step - loss: 1.6382 - categorical_accuracy: 0.4312 - val_loss: 1.6461 - val_categorical_accuracy: 0.4232
Epoch 198/200
98/98 [==============================] - 6s 62ms/step - loss: 1.6372 - categorical_accuracy: 0.4311 - val_loss: 1.6420 - val_categorical_accuracy: 0.4263
Epoch 199/200
98/98 [==============================] - 6s 62ms/step - loss: 1.6369 - categorical_accuracy: 0.4307 - val_loss: 1.6387 - val_categorical_accuracy: 0.4306
Epoch 200/200
98/98 [==============================] - 6s 62ms/step - loss: 1.6336 - categorical_accuracy: 0.4325 - val_loss: 1.6443 - val_categorical_accuracy: 0.4259


##### Optimizer 
SGD ADAM RMSprop

In [19]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = Adam
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

Epoch 1/200
98/98 [==============================] - 5s 49ms/step - loss: 2.2884 - categorical_accuracy: 0.1265 - val_loss: 2.2727 - val_categorical_accuracy: 0.1458
Epoch 2/200
98/98 [==============================] - 5s 46ms/step - loss: 2.2605 - categorical_accuracy: 0.1625 - val_loss: 2.2475 - val_categorical_accuracy: 0.1881
Epoch 3/200
98/98 [==============================] - 5s 46ms/step - loss: 2.2352 - categorical_accuracy: 0.1940 - val_loss: 2.2229 - val_categorical_accuracy: 0.2107
Epoch 4/200
98/98 [==============================] - 5s 46ms/step - loss: 2.2100 - categorical_accuracy: 0.2156 - val_loss: 2.1980 - val_categorical_accuracy: 0.2151
Epoch 5/200
98/98 [==============================] - 5s 46ms/step - loss: 2.1852 - categorical_accuracy: 0.2237 - val_loss: 2.1740 - val_categorical_accuracy: 0.2276
Epoch 6/200
98/98 [==============================] - 5s 46ms/step - loss: 2.1613 - categorical_accuracy: 0.2343 - val_loss: 2.1515 - val_categorical_accuracy: 0.2379
Epoc

98/98 [==============================] - 4s 44ms/step - loss: 1.6334 - categorical_accuracy: 0.4361 - val_loss: 1.6390 - val_categorical_accuracy: 0.4218
Epoch 100/200
98/98 [==============================] - 4s 45ms/step - loss: 1.6305 - categorical_accuracy: 0.4369 - val_loss: 1.6371 - val_categorical_accuracy: 0.4260
Epoch 101/200
98/98 [==============================] - 4s 44ms/step - loss: 1.6284 - categorical_accuracy: 0.4383 - val_loss: 1.6296 - val_categorical_accuracy: 0.4288
Epoch 102/200
98/98 [==============================] - 4s 44ms/step - loss: 1.6258 - categorical_accuracy: 0.4401 - val_loss: 1.6301 - val_categorical_accuracy: 0.4285
Epoch 103/200
98/98 [==============================] - 4s 44ms/step - loss: 1.6229 - categorical_accuracy: 0.4389 - val_loss: 1.6241 - val_categorical_accuracy: 0.4314
Epoch 104/200
98/98 [==============================] - 4s 45ms/step - loss: 1.6209 - categorical_accuracy: 0.4403 - val_loss: 1.6222 - val_categorical_accuracy: 0.4309
Epoch 

Epoch 148/200
98/98 [==============================] - 4s 45ms/step - loss: 1.5264 - categorical_accuracy: 0.4688 - val_loss: 1.5315 - val_categorical_accuracy: 0.4614
Epoch 149/200
98/98 [==============================] - 4s 44ms/step - loss: 1.5245 - categorical_accuracy: 0.4698 - val_loss: 1.5350 - val_categorical_accuracy: 0.4600
Epoch 150/200
98/98 [==============================] - 4s 44ms/step - loss: 1.5230 - categorical_accuracy: 0.4699 - val_loss: 1.5279 - val_categorical_accuracy: 0.4606
Epoch 151/200
98/98 [==============================] - 4s 44ms/step - loss: 1.5205 - categorical_accuracy: 0.4706 - val_loss: 1.5280 - val_categorical_accuracy: 0.4628
Epoch 152/200
98/98 [==============================] - 4s 45ms/step - loss: 1.5201 - categorical_accuracy: 0.4709 - val_loss: 1.5313 - val_categorical_accuracy: 0.4567
Epoch 153/200
98/98 [==============================] - 4s 44ms/step - loss: 1.5181 - categorical_accuracy: 0.4715 - val_loss: 1.5256 - val_categorical_accuracy:

Epoch 197/200
98/98 [==============================] - 4s 45ms/step - loss: 1.4638 - categorical_accuracy: 0.4875 - val_loss: 1.4778 - val_categorical_accuracy: 0.4794
Epoch 198/200
98/98 [==============================] - 4s 44ms/step - loss: 1.4628 - categorical_accuracy: 0.4892 - val_loss: 1.4780 - val_categorical_accuracy: 0.4795
Epoch 199/200
98/98 [==============================] - 4s 45ms/step - loss: 1.4615 - categorical_accuracy: 0.4898 - val_loss: 1.4788 - val_categorical_accuracy: 0.4812
Epoch 200/200
98/98 [==============================] - 4s 44ms/step - loss: 1.4613 - categorical_accuracy: 0.4898 - val_loss: 1.4813 - val_categorical_accuracy: 0.4760


In [20]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = RMSprop
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=RMSprop(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet", f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

Epoch 1/200
98/98 [==============================] - 4s 37ms/step - loss: 2.3123 - categorical_accuracy: 0.3353 - val_loss: 1.5106 - val_categorical_accuracy: 0.4594
Epoch 2/200
98/98 [==============================] - 4s 36ms/step - loss: 1.2802 - categorical_accuracy: 0.5466 - val_loss: 1.1771 - val_categorical_accuracy: 0.5861
Epoch 3/200
98/98 [==============================] - 3s 35ms/step - loss: 1.0375 - categorical_accuracy: 0.6360 - val_loss: 1.0499 - val_categorical_accuracy: 0.6323
Epoch 4/200
98/98 [==============================] - 3s 35ms/step - loss: 0.8559 - categorical_accuracy: 0.7036 - val_loss: 1.0531 - val_categorical_accuracy: 0.6397
Epoch 5/200
98/98 [==============================] - 3s 36ms/step - loss: 0.7437 - categorical_accuracy: 0.7416 - val_loss: 1.0636 - val_categorical_accuracy: 0.6439
Epoch 6/200
98/98 [==============================] - 3s 35ms/step - loss: 0.6173 - categorical_accuracy: 0.7870 - val_loss: 1.1299 - val_categorical_accuracy: 0.6414
Epoc

Epoch 99/200
98/98 [==============================] - 4s 36ms/step - loss: 0.0824 - categorical_accuracy: 0.9869 - val_loss: 14.8812 - val_categorical_accuracy: 0.5954
Epoch 100/200
98/98 [==============================] - 4s 36ms/step - loss: 0.0809 - categorical_accuracy: 0.9879 - val_loss: 15.1362 - val_categorical_accuracy: 0.5875
Epoch 101/200
98/98 [==============================] - 3s 36ms/step - loss: 0.1052 - categorical_accuracy: 0.9847 - val_loss: 15.1641 - val_categorical_accuracy: 0.5920
Epoch 102/200
98/98 [==============================] - 4s 36ms/step - loss: 0.0843 - categorical_accuracy: 0.9884 - val_loss: 14.8994 - val_categorical_accuracy: 0.6032
Epoch 103/200
98/98 [==============================] - 4s 36ms/step - loss: 0.1021 - categorical_accuracy: 0.9855 - val_loss: 15.7354 - val_categorical_accuracy: 0.5940
Epoch 104/200
98/98 [==============================] - 4s 36ms/step - loss: 0.0823 - categorical_accuracy: 0.9873 - val_loss: 14.9815 - val_categorical_accu

98/98 [==============================] - 3s 34ms/step - loss: 0.1211 - categorical_accuracy: 0.9911 - val_loss: 34.0367 - val_categorical_accuracy: 0.5942
Epoch 196/200
98/98 [==============================] - 3s 33ms/step - loss: 0.1002 - categorical_accuracy: 0.9923 - val_loss: 32.9339 - val_categorical_accuracy: 0.5901
Epoch 197/200
98/98 [==============================] - 3s 33ms/step - loss: 0.1080 - categorical_accuracy: 0.9923 - val_loss: 33.8506 - val_categorical_accuracy: 0.5854
Epoch 198/200
98/98 [==============================] - 3s 33ms/step - loss: 0.1226 - categorical_accuracy: 0.9910 - val_loss: 32.3865 - val_categorical_accuracy: 0.5976
Epoch 199/200
98/98 [==============================] - 3s 33ms/step - loss: 0.1140 - categorical_accuracy: 0.9917 - val_loss: 35.2252 - val_categorical_accuracy: 0.5876
Epoch 200/200
98/98 [==============================] - 3s 33ms/step - loss: 0.1112 - categorical_accuracy: 0.9918 - val_loss: 37.1400 - val_categorical_accuracy: 0.5913


#### Normalisation

In [21]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax,kernel_regularizer=l2(0.5), bias_regularizer=l2(0.5))(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet", f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.5")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.5.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

Epoch 1/200
98/98 [==============================] - 5s 47ms/step - loss: 7.1352 - categorical_accuracy: 0.2123 - val_loss: 3.5519 - val_categorical_accuracy: 0.2663
Epoch 2/200
98/98 [==============================] - 4s 44ms/step - loss: 2.6811 - categorical_accuracy: 0.2840 - val_loss: 2.2281 - val_categorical_accuracy: 0.3058
Epoch 3/200
98/98 [==============================] - 4s 44ms/step - loss: 2.1011 - categorical_accuracy: 0.3119 - val_loss: 2.0416 - val_categorical_accuracy: 0.3249
Epoch 4/200
98/98 [==============================] - 5s 47ms/step - loss: 2.0077 - categorical_accuracy: 0.3228 - val_loss: 1.9843 - val_categorical_accuracy: 0.3355
Epoch 5/200
98/98 [==============================] - 5s 48ms/step - loss: 1.9806 - categorical_accuracy: 0.3263 - val_loss: 1.9622 - val_categorical_accuracy: 0.3291
Epoch 6/200
98/98 [==============================] - 5s 47ms/step - loss: 1.9528 - categorical_accuracy: 0.3400 - val_loss: 1.9524 - val_categorical_accuracy: 0.3239
Epoc

98/98 [==============================] - 5s 47ms/step - loss: 1.5483 - categorical_accuracy: 0.4792 - val_loss: 1.5547 - val_categorical_accuracy: 0.4671
Epoch 100/200
98/98 [==============================] - 5s 47ms/step - loss: 1.5487 - categorical_accuracy: 0.4783 - val_loss: 1.5967 - val_categorical_accuracy: 0.4466
Epoch 101/200
98/98 [==============================] - 5s 47ms/step - loss: 1.5490 - categorical_accuracy: 0.4775 - val_loss: 1.5428 - val_categorical_accuracy: 0.4718
Epoch 102/200
98/98 [==============================] - 5s 47ms/step - loss: 1.5489 - categorical_accuracy: 0.4758 - val_loss: 1.5455 - val_categorical_accuracy: 0.4803
Epoch 103/200
98/98 [==============================] - 5s 47ms/step - loss: 1.5512 - categorical_accuracy: 0.4755 - val_loss: 1.5362 - val_categorical_accuracy: 0.4773
Epoch 104/200
98/98 [==============================] - 5s 47ms/step - loss: 1.5401 - categorical_accuracy: 0.4814 - val_loss: 1.5348 - val_categorical_accuracy: 0.4848
Epoch 

Epoch 148/200
98/98 [==============================] - 5s 47ms/step - loss: 1.4952 - categorical_accuracy: 0.4974 - val_loss: 1.4879 - val_categorical_accuracy: 0.5002
Epoch 149/200
98/98 [==============================] - 5s 47ms/step - loss: 1.4949 - categorical_accuracy: 0.4978 - val_loss: 1.5024 - val_categorical_accuracy: 0.4938
Epoch 150/200
98/98 [==============================] - 5s 47ms/step - loss: 1.5056 - categorical_accuracy: 0.4949 - val_loss: 1.4780 - val_categorical_accuracy: 0.5057
Epoch 151/200
98/98 [==============================] - 5s 47ms/step - loss: 1.4994 - categorical_accuracy: 0.4993 - val_loss: 1.4751 - val_categorical_accuracy: 0.5030
Epoch 152/200
98/98 [==============================] - 5s 47ms/step - loss: 1.4924 - categorical_accuracy: 0.4977 - val_loss: 1.5087 - val_categorical_accuracy: 0.4890
Epoch 153/200
98/98 [==============================] - 5s 48ms/step - loss: 1.4981 - categorical_accuracy: 0.4974 - val_loss: 1.5010 - val_categorical_accuracy:

Epoch 197/200
98/98 [==============================] - 4s 44ms/step - loss: 1.4595 - categorical_accuracy: 0.5133 - val_loss: 1.4705 - val_categorical_accuracy: 0.4992
Epoch 198/200
98/98 [==============================] - 4s 44ms/step - loss: 1.4685 - categorical_accuracy: 0.5079 - val_loss: 1.4680 - val_categorical_accuracy: 0.5122
Epoch 199/200
98/98 [==============================] - 4s 44ms/step - loss: 1.4381 - categorical_accuracy: 0.5228 - val_loss: 1.4996 - val_categorical_accuracy: 0.4921
Epoch 200/200
98/98 [==============================] - 4s 44ms/step - loss: 1.4618 - categorical_accuracy: 0.5131 - val_loss: 1.4483 - val_categorical_accuracy: 0.5196


In [22]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 200
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9

def ResNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        prev_layer = hidden_layers
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding='same', activation=relu,
                                            kernel_initializer=he_normal)(hidden_layers)
        #hidden_layers = Dropout(0.2)(hidden_layers)
        if n > 0:
            hidden_layers = Average()([hidden_layers, prev_layer])
        else:
            hidden_layers = Average()([hidden_layers, Dense(32)(prev_layer)])
    
    hidden_layers = Flatten()(hidden_layers)   
    output_layer = Dense(NUM_CLASSES, activation=softmax,kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32 for x in range(1, num_layer+1)]

resnet = ResNet(num_layer, filters_by_layers)
resnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(learning_rates, momentum=momentums),
            metrics=categorical_accuracy)
RESNET_LOG = os.path.join(LOG_DIR, "resnet", f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.01")
RESNET_MODEL = os.path.join(MODEL_DIR, "resnet",
                f"resnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{learning_rates}_mom_{momentums}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu_l20.01.keras")
resnet.fit(x_train,
                y_train,
                batch_size=BATCH_SIZE,
                epochs=EPOCHS,
                validation_data=(x_test, y_test),
                shuffle=SHUFFLE,
                callbacks=[tf.keras.callbacks.TensorBoard(RESNET_LOG, histogram_freq=1)]
               ) 
resnet.save(RESNET_MODEL)

Epoch 1/200
98/98 [==============================] - 5s 47ms/step - loss: 2.4167 - categorical_accuracy: 0.1827 - val_loss: 2.3365 - val_categorical_accuracy: 0.2222
Epoch 2/200
98/98 [==============================] - 4s 44ms/step - loss: 2.2892 - categorical_accuracy: 0.2539 - val_loss: 2.2477 - val_categorical_accuracy: 0.2859
Epoch 3/200
98/98 [==============================] - 4s 44ms/step - loss: 2.1987 - categorical_accuracy: 0.3032 - val_loss: 2.1540 - val_categorical_accuracy: 0.3246
Epoch 4/200
98/98 [==============================] - 4s 44ms/step - loss: 2.1040 - categorical_accuracy: 0.3350 - val_loss: 2.0610 - val_categorical_accuracy: 0.3483
Epoch 5/200
98/98 [==============================] - 4s 44ms/step - loss: 2.0466 - categorical_accuracy: 0.3509 - val_loss: 2.0253 - val_categorical_accuracy: 0.3559
Epoch 6/200
98/98 [==============================] - 4s 45ms/step - loss: 2.0134 - categorical_accuracy: 0.3600 - val_loss: 1.9966 - val_categorical_accuracy: 0.3614
Epoc

98/98 [==============================] - 4s 44ms/step - loss: 1.1563 - categorical_accuracy: 0.6170 - val_loss: 1.2987 - val_categorical_accuracy: 0.5610
Epoch 100/200
98/98 [==============================] - 4s 44ms/step - loss: 1.1586 - categorical_accuracy: 0.6153 - val_loss: 1.2868 - val_categorical_accuracy: 0.5637
Epoch 101/200
98/98 [==============================] - 4s 44ms/step - loss: 1.1485 - categorical_accuracy: 0.6203 - val_loss: 1.2718 - val_categorical_accuracy: 0.5717
Epoch 102/200
98/98 [==============================] - 4s 44ms/step - loss: 1.1473 - categorical_accuracy: 0.6211 - val_loss: 1.2942 - val_categorical_accuracy: 0.5619
Epoch 103/200
98/98 [==============================] - 4s 44ms/step - loss: 1.1436 - categorical_accuracy: 0.6205 - val_loss: 1.2682 - val_categorical_accuracy: 0.5724
Epoch 104/200
98/98 [==============================] - 4s 44ms/step - loss: 1.1342 - categorical_accuracy: 0.6268 - val_loss: 1.2770 - val_categorical_accuracy: 0.5729
Epoch 

Epoch 148/200
98/98 [==============================] - 4s 44ms/step - loss: 1.0084 - categorical_accuracy: 0.6791 - val_loss: 1.2533 - val_categorical_accuracy: 0.5934
Epoch 149/200
98/98 [==============================] - 4s 44ms/step - loss: 1.0032 - categorical_accuracy: 0.6818 - val_loss: 1.2641 - val_categorical_accuracy: 0.5882
Epoch 150/200
98/98 [==============================] - 4s 44ms/step - loss: 0.9962 - categorical_accuracy: 0.6848 - val_loss: 1.2157 - val_categorical_accuracy: 0.6070
Epoch 151/200
98/98 [==============================] - 4s 44ms/step - loss: 0.9971 - categorical_accuracy: 0.6830 - val_loss: 1.2186 - val_categorical_accuracy: 0.6085
Epoch 152/200
98/98 [==============================] - 4s 44ms/step - loss: 0.9932 - categorical_accuracy: 0.6826 - val_loss: 1.2543 - val_categorical_accuracy: 0.5922
Epoch 153/200
98/98 [==============================] - 4s 44ms/step - loss: 1.0083 - categorical_accuracy: 0.6787 - val_loss: 1.2125 - val_categorical_accuracy:

Epoch 197/200
98/98 [==============================] - 4s 44ms/step - loss: 0.8589 - categorical_accuracy: 0.7437 - val_loss: 1.2065 - val_categorical_accuracy: 0.6290
Epoch 198/200
98/98 [==============================] - 4s 44ms/step - loss: 0.8671 - categorical_accuracy: 0.7385 - val_loss: 1.2276 - val_categorical_accuracy: 0.6182
Epoch 199/200
98/98 [==============================] - 4s 44ms/step - loss: 0.8471 - categorical_accuracy: 0.7474 - val_loss: 1.2933 - val_categorical_accuracy: 0.5992
Epoch 200/200
98/98 [==============================] - 4s 44ms/step - loss: 0.8766 - categorical_accuracy: 0.7326 - val_loss: 1.2020 - val_categorical_accuracy: 0.6301
